<a href="https://colab.research.google.com/github/SoniaSun810/LangChain_Practice/blob/main/LangChain_%2B_Pinecone_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langchain openai pymysql --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''

# LangChain SQl Database Agent

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import OpenAI
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
db_user = "admin"
db_password = "MSWE-ta-project!"
db_host = "aws-ta-matching.cpxfig1ppqqw.eu-north-1.rds.amazonaws.com"
db_name = "TA_Matching_System"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")

In [ ]:
toolkit = SQLDatabaseToolkit(db = db, llm = llm)

In [ ]:
agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    verbose = True
)

# Pinecone + LangChain

In [ ]:
pip install pinecone-client openai tiktoken
pip install pinecone-client

In [ ]:
import os
#load .env file
from dotenv import load_dotenv
load_dotenv()
from langchain import OpenAI
# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_environment= os.getenv("PINECONE_API_ENV")
print(openai_api_key)
print(pinecone_api_key)
print(pinecone_environment)

In [ ]:
#importing langChain document loaders and text processors
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = CSVLoader(file_path='MSWE TA Matching System Summer 2023 - Student Info(Filtered Fields).csv')
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts = text_splitter.split_documents(data)
print(data)
print(texts)

In [ ]:
#importing pinecone packages
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
#initialize pinecone with api key and environment
pinecone.init(
    api_key= pinecone_api_key,
    environment=pinecone_environment
)
index_name = "students"

In [ ]:
#recursive document search function
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
#establishing llm and the QA Chain
llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
#declaring the query variable
#declaring the docs variable
query = "Which 5 students are suitable Teaching assistant candidates for this course: I&C SCI 9	Introduction to Computation for Scientists and Engineers	Introduces principles, techniques, and computational tools for quantitative approach to basic problem solving in physics and engineering. Pre-requisites: MATH 2A or AP Calculus AB or AP Calculus BC. Overlaps: Physics 2 "
docs = docsearch.similarity_search(query)

In [ ]:
#running the chain and passing parameters: the input and the question
chain.run(input_documents=docs, question=query)